In [ ]:

import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, LSTM, Dropout, SimpleRNN
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

np.random.seed(42)
tf.random.set_seed(42)

DATA_DIR = r"C:\Users\Jeet\Desktop\IIT_Gandhinagar\Dataset_Folder\UCI HAR Dataset\UCI HAR Dataset"

def load_inertial_data(folder):
    """
    Load inertial sensor data from the specified folder (train/test).
    """
    signals = ['body_acc_x', 'body_acc_y', 'body_acc_z',
               'body_gyro_x', 'body_gyro_y', 'body_gyro_z']
    
    data = []
    for signal in signals:
        file_path = os.path.join(DATA_DIR, folder, 'Inertial Signals', f"{signal}_{folder}.txt")
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    return np.stack(data, axis=-1)  

def load_labels(folder):
    """
    Load activity labels for train/test data.
    """
    file_path = os.path.join(DATA_DIR, folder, 'y_' + folder + '.txt')
    return np.loadtxt(file_path, dtype=int)


print("Loading data...")
X_train = load_inertial_data('train')
y_train = load_labels('train')
X_test = load_inertial_data('test')
y_test = load_labels('test')

print(f"Train data shape: {X_train.shape}, Train labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}, Test labels shape: {y_test.shape}")

# Preprocessing Labels
from tensorflow.keras.utils import to_categorical
y_train_ohe = to_categorical(y_train - 1)  # Adjust labels to start from 0
y_test_ohe = to_categorical(y_test - 1)

# Defining model specficications
def create_1d_cnn(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        Conv1D(64, kernel_size=3, activation='relu'),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

def create_mlp(input_shape, num_classes):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

def create_lstm(input_shape, num_classes):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        LSTM(32),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model


def evaluate_model(model_func, X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies, precisions, recalls, f1s = [], [], [], []
    
    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        model = model_func(X_train_fold.shape[1:], y_train_fold.shape[1])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)
        
        y_pred = model.predict(X_val_fold)
        y_pred_labels = np.argmax(y_pred, axis=1)
        y_true_labels = np.argmax(y_val_fold, axis=1)
        
        acc = accuracy_score(y_true_labels, y_pred_labels)
        prec = precision_score(y_true_labels, y_pred_labels, average='weighted')
        rec = recall_score(y_true_labels, y_pred_labels, average='weighted')
        f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
        
        accuracies.append(acc)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
    
    print("Average Accuracy:", np.mean(accuracies))
    print("Average Precision:", np.mean(precisions))
    print("Average Recall:", np.mean(recalls))
    print("Average F1 Score:", np.mean(f1s))

print("\nEvaluating 1D-CNN Model...")
evaluate_model(create_1d_cnn, X_train, y_train_ohe)

print("\nEvaluating MLP Model...")
evaluate_model(create_mlp, X_train, y_train_ohe)

print("\nEvaluating LSTM Model...")
evaluate_model(create_lstm, X_train, y_train_ohe)


Loading data...
Train data shape: (7352, 128, 6), Train labels shape: (7352,)
Test data shape: (2947, 128, 6), Test labels shape: (2947,)

Evaluating 1D-CNN Model...


C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


...

C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step


C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step
Average Accuracy: 0.5066835925396671
Average Precision: 0.46193497620805324
Average Recall: 0.5066835925396671
Average F1 Score: 0.43195060435330446


C:\Users\Jeet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
